### Programação Linear 

*Lista IV - Estudo de Caso*

Guilherme Cadori

01/07/2023

##### Ex. 1) Exemplo Acadêmico do Artigo Selecionado

A seguir será apresentado uma aplicação de dimensões reduzidas da abordagem de modelagem apresentada no artigo “Spatial optimization of multiple area land acquisition”. O artigo foi desenvolvido pro Alan T. Murray e Richard L. Church, pesquisadores da Universidade da Califórnia, EUA, e publicado no aclamado periódico *Computers and Operations Research*.

A aplicação apresentada a seguir foi reduzidas para dimensões administráveis, com uma quantidade reduzida de variáveis, para finalidades acadêmicas. O dados utilizados foram todos gerados para esta aplicação especificamente e serão apresentados a seguir.


In [296]:
# Iportando biblioteca de trabalho
import gurobipy as gp


In [297]:
# Configurando parâmetros

# Quantidade de áreas a serem formadas
p = 4

# Tamanho máximo para área a ser formada
T = 50

# Criando dados as unidades a serem avaliadas
# Assumindo que cada área terá um tamanho de 10 acres
indiceUnEspacial, beneficio, area = gp.multidict({
    "1": [24, 10], "2": [29, 10], "3": [19, 10], "4": [35, 10], 
    "5": [24, 10], "6": [19, 10], "7": [33, 10], "8": [29, 10], 
    "9": [26, 10], "10": [21, 10], "11": [18, 10], "12": [21, 10],
    "13": [31, 10], "14": [33, 10], "15": [31, 10], "16": [32, 10]
})


In [298]:
# Criando o modelo
m = gp.Model("Otimização Espacial")


## Criando variáveis de decisão
# **** 
#    Obs: esse problema possui múltiplas classes de variáveis de decisão
#         e parte dessas não fazer parte da função objetivo
# ****

# Unidades espaciais - Xi
unidadeEspacial = m.addVars(indiceUnEspacial, 
                            vtype = gp.GRB.BINARY,
                            name = "Unidade Espacial")

# Unidade de área selecionada como "sink" (direcionadoras de fluxo) - Vi
unidadeEspacialSink = m.addVars(indiceUnEspacial, 
                                vtype = gp.GRB.BINARY,
                                name = "Unidade Espacial Sink")

# Fluxo de i para j direcionada ao sink - Yij
fluxoEspacialSink = m.addVars(unidadeEspacial, unidadeEspacial, 
                              vtype = gp.GRB.CONTINUOUS, 
                              name = "Fluxo Espacial Sink")

# Arco entre i e j utilizado para direcionar o fluxo ao sink - Yij_Arc
fluxoEspacialArc = m.addVars(unidadeEspacial, unidadeEspacial, 
                             vtype = gp.GRB.BINARY, 
                             name = "Arco do Fluxo Espacial Sink")

In [299]:
# Conferindo variáveis criadas
# Ps. Remover "#" e rodar um-a-um

# unidadeEspacial

# unidadeEspacialSink

# fluxoEspacialSink

# fluxoEspacialArc


In [300]:
# Criando relações de vizinhaça entre as unidades avaliadas
vizinhanca = {
    ("1", "2"): 1, ("1", "5"): 1, 
    ("2", "1"): 1, ("2", "3"): 1, ("2", "6"): 1, 
    ("3", "2"): 1, ("3", "4"): 1, ("3", "7"): 1,
    ("4", "3"): 1, ("4", "8"): 1,
    ("5", "1"): 1, ("5", "6"): 1, ("5", "9"): 1,
    ("6", "2"): 1, ("6", "5"): 1, ("6", "7"): 1,   ("6", "10"): 1,
    ("7", "3"): 1, ("7", "6"): 1, ("7", "8"): 1,   ("7", "11"): 1,
    ("8", "4"): 1, ("8", "7"): 1, ("8", "12"): 1, 
    ("9", "5"): 1, ("9", "10"): 1, ("9", "13"): 1, 
    ("10", "6"): 1, ("10", "9"): 1, ("10", "11"): 1, ("10", "14"): 1,
    ("11", "7"): 1, ("11", "10"): 1, ("11", "12"): 1, ("11", "15"): 1,
    ("12", "8"): 1, ("12", "11"): 1, ("12", "16"): 1,
    ("13", "9"): 1, ("13", "14"): 1,
    ("14", "10"): 1, ("14", "13"): 1, ("14", "15"): 1,
    ("15", "11"): 1, ("15", "14"): 1, ("15", "16"): 1,
    ("16", "12"): 1, ("16", "15"): 1
}

vizinhanca_complete = {}

for i in range(1, 17):
    for j in range(1, 17):
        if (str(i), str(j)) in vizinhanca or (str(j), str(i)) in vizinhanca:
            vizinhanca_complete[(str(i), str(j))] = 1
        else:
            vizinhanca_complete[(str(i), str(j))] = 0



In [301]:
# Criando a função objetivo (1)
m.setObjective(unidadeEspacial.prod(beneficio), gp.GRB.MAXIMIZE)


In [302]:
# Criando a restrição (2)
rest2 = m.addConstrs((gp.quicksum(fluxoEspacialSink[i, j] for j in unidadeEspacial) 
                          - gp.quicksum(fluxoEspacialSink[j, i] for j in unidadeEspacial) 
                          >= beneficio[i] * unidadeEspacial[i] - T * unidadeEspacialSink[i]
                          for i in unidadeEspacial), 
                          name = "Restrição (2)")


In [303]:
# Criando a restrição (3)
constraint = m.addConstrs((fluxoEspacialSink[i, j] <= T * fluxoEspacialArc[i, j]
                          for i in unidadeEspacial for j in unidadeEspacial),
                          name = "Restrição (3)")


In [304]:
# Criando a restrição (4)
constraint = m.addConstrs((sum(fluxoEspacialArc[i, j] for j in unidadeEspacial) <= 1
                          for i in unidadeEspacial),
                          name = "Restrição (4)")


In [305]:
# Criando a restrição (5)
constraint = m.addConstrs((T * unidadeEspacialSink[i] + 
                           sum(fluxoEspacialSink[i, j] for j in unidadeEspacial) 
                           <= T for i in unidadeEspacial),
                           name="Restrição (5)")


In [306]:
# Criando a restrição (6)
rest6 = m.addConstrs((sum(unidadeEspacialSink[i] for i in indiceUnEspacial)
                      == p for j in indiceUnEspacial), 
                      name = "Restrição (6)")


In [307]:
# Criando a restrição (7)
constraint = m.addConstrs((sum(fluxoEspacialSink[i, j] for j in unidadeEspacial) 
                           <= T * unidadeEspacial[i] for i in unidadeEspacial),
                           name="Restrição (7)")


In [308]:
# Criando a restrição (8)
rest8 = m.addConstrs((unidadeEspacialSink[i] <= unidadeEspacial[i] 
                      for i in indiceUnEspacial), 
                      name="Restrição (8)")


In [309]:
# Otimizando o modelo
m.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 352 rows, 544 columns and 2112 nonzeros
Model fingerprint: 0xe6197a3e
Variable types: 256 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 110.0000000
Presolve removed 79 rows and 48 columns
Presolve time: 0.01s
Presolved: 273 rows, 496 columns, 1488 nonzeros
Variable types: 240 continuous, 256 integer (256 binary)

Root relaxation: objective 2.000000e+02, 100 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [310]:
# Conferindo o valor da função objetivo
print("Valor Objetivo:", m.objVal)

# # Conferindo as variáveis de decisão
# for var in m.getVars():
#     if var.X > 0:
#         print(f"{var.VarName}: {var.X}")
        
# Iterate over the variables and print non-zero values
for var in m.getVars():
    if var.x > 0 and (var.varName.startswith("Unidade Espacial") or var.varName.startswith("Unidade Espacial Sink")):
        print(var.varName,": ", var.x, sep="")


Valor Objetivo: 200.0
Unidade Espacial[1]: 1.0
Unidade Espacial[2]: 1.0
Unidade Espacial[8]: 1.0
Unidade Espacial[9]: 1.0
Unidade Espacial[10]: 1.0
Unidade Espacial[11]: 1.0
Unidade Espacial[12]: 1.0
Unidade Espacial[16]: 1.0
Unidade Espacial Sink[1]: 1.0
Unidade Espacial Sink[2]: 1.0
Unidade Espacial Sink[8]: 1.0
Unidade Espacial Sink[11]: 1.0


### Fim do Exemplo